<center><h2>Using find address candidates API geocoding</h2></center>

<p>reference: https://developers.arcgis.com/rest/geocode/api-reference/geocoding-find-address-candidates.htm <br>
 url: http://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer/findAddressCandidates?f=pjson&outFields=Subregion&forStorage=false&SingleLine={}<br>
example result<json format>: <br>
    {
 "spatialReference": {
  "wkid": 4326,
  "latestWkid": 4326
 },<br>
 "candidates": [<br>
  {
   "address": "Pacific Hwy & W Washington St, San Diego, California, 92140",
   "location": {
    "x": -117.18432899999999,
    "y": 32.740442999999999
   },<br>
   "score": 100,
   "attributes": {
    "Subregion": "San Diego County"
   },<br>
   "extent": {
    "xmin": -117.185329,
    "ymin": 32.739443000000001,
    "xmax": -117.18332899999999,
    "ymax": 32.741442999999997
   }<br>
  }
 ]
}
    </p>
<p>Batch geocode access
An ArcGIS Online organizational account is required to use the batch geocoding functionality provided by the World Geocoding Service. Successfully geocoded addresses that return a status of Matched or Tied cause ArcGIS Online service credits to be consumed for batch geocoding operations.

Each batch geocode request requires a token to be included with the request. Refer to Authenticate a request to the World Geocoding Service for information about how to access secured batch geocoding functionality.</p>
<p>Using cura geocode example(responsed result didn't contain county name):
    http://cura-gis-web.asc.ohio-state.edu/arcgis/rest/services/geocoding/North_America_Locator_2015/GeocodeServer/geocodeAddresses?addresses=%7B+++++++++++%22records%22%3A+%5B%0D%0A++++++++%7B%0D%0A++++++++++++%22attributes%22%3A+%7B%0D%0A++++++++++++++++%22OBJECTID%22%3A+1%2C%0D%0A++++++++++++++++%22Street%22%3A+%22380+New+York+St.%22%2C%0D%0A++++++++++++++++%22City%22%3A+%22Redlands%22%2C%0D%0A++++++++++++++++%22State%22%3A+%22CA%22%2C%0D%0A++++++++++++++++%22ZIP%22%3A+%2292373%22%0D%0A++++++++++++%7D%0D%0A++++++++%7D%2C%0D%0A+++%7B%0D%0A++++++++++++%22attributes%22%3A+%7B%0D%0A++++++++++++++++%22OBJECTID%22%3A+2%2C%0D%0A++++++++++++++++%22Street%22%3A+%221+World+Way%22%2C%0D%0A++++++++++++++++%22City%22%3A+%22Los+Angeles%22%2C%0D%0A++++++++++++++++%22State%22%3A+%22CA%22%2C%0D%0A++++++++++++++++%22ZIP%22%3A+%2290045%22%0D%0A++++++++++++%7D%0D%0A++++++++%7D%0D%0A++++%5D%0D%0A%7D%0D%0A&category=&sourceCountry=United+States&outSR=&f=pjson
    </p>

In [1]:
import requests
import json

In [2]:
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize

In [3]:
df = pd.read_excel('GreenSpot Businesses Raw Data.xlsx')
df.head()


,Industry,BusinessName,Id,FirstName,Email,LastName,Address,City,State,Zip,EnrollmentDate
0,office_and_education,Clevest Solutions Inc,9641,Jared,jared.brandon@clevest.com,Brandon,#100 - 13911 Wireless Way,Richmond,OH,43215,2010-10-17
1,industrial,Abbott Nutrition,10112,Susan,susan.davis@abbott.com,Davis,1 abbott park way,tipp city,OH,45371,2013-06-24
2,office_and_education,MAPFRE Stadium,10407,Jordan,jenke@columbuscrewsc.com,Enke,1 Black and Gold Blvd,Columbus,OH,43211,2015-04-21
3,restaurant_and_grocery,Cathy Tiffany Salon Lofts,9446,Cathy,cathy@cathytiffany.com,Tiffany,1 E. Campus View Blvd,Columbus,OH,43235,2008-08-02
4,office_and_education,Nationwide Insurance,9728,Kelly,grosenk1@nationwide.com,Grosenbacher,1 Nationwide Plaza,Columbus,OH,43215,2009-10-19


In [34]:
df['Address'] = df.Address.astype(str)
#attern = '|'.join(['#', '.', ','])
df['Address'] = df.Address.str.replace(' ','%').str.strip()
df['City'] = df.City.astype(str)
#attern = '|'.join(['#', '.', ','])
df['City'] = df.City.str.replace(' ','%').str.strip()

In [43]:
df_select = df.sample(n=100, random_state=1).copy()
df_select.reset_index(inplace = True)
df_select.head()

,index,Industry,BusinessName,Id,FirstName,Email,LastName,Address,City,State,Zip,EnrollmentDate
0,201,office_and_education,Association of College and University Housing ...,10438,Shaun,shaun@acuho-i.org,Holloway,1445%Summit%Street,Columbus,OH,43201,2015-04-30 00:00:00
1,115,office_and_education,"OSU - Kinnear Rd, 1275",10165,Aparna,rrdenoewer@columbus.gov,Dial,1275%Kinnear%Rd.,Columbus,OH,43212,2013-07-24 00:00:00
2,255,industrial,Frog Hauling Dumpster Rental,10025,Marty,info@froghauling.com,Reed,1601%West%5th.%Avenue%#127,Columbus,OH,43212,2010-09-08 00:00:00
3,1103,office_and_education,Wild Ones Columbus Chapter,10525,Barbara K,bkvelezbarbosa@gmail.com,Velez Barbosa,7292%Basil%Western%Road,Canal%Winchester,OH,43110,2015-10-22 00:00:00
4,195,office_and_education,Huckleberry House,11968,Becky,rwesterfelt@huck-house.org,Westerfelt,1421%Hamlet%Street,Columbus,OH,43204,2016-06-30 12:47:27


In [65]:
#eg: single line looks like: 
def geocode(row):
    area='-84.767215185999945%2C38.357056799000063%2C-80.65521518599995%2C42.469056799000057'
    try:
        url= "http://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer/findAddressCandidates?f=json&searchExtent=area&maxLocations=2&outFields=Subregion&forStorage=false&Address={}&City={}&Region={}&Postal={}".format(row['Address'],row['City'],row['State'],row['Zip'])
        #url= "http://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer/findAddressCandidates?f=json&searchExtent=area&maxLocations=2&outFields=*&forStorage=false&Address={}&City={}&Region={}&Postal={}".format(row['Address'],row['City'],row['State'],row['Zip'])
        
        response = requests.get(url)
        #print(response.json)
    except requests.exceptions:
        return ''
    else:
        data = response.json()
        if (response.status_code == 200)and (data['candidates'] != []):
            
            return data["candidates"][0]['attributes']['Subregion']
            #return data
        else:
            return ''


In [66]:
import timeit

start = timeit.default_timer()
df['county'] = df.apply(geocode,axis=1)
stop = timeit.default_timer()

print('Time: ', stop - start) 

Time:  739.6122185000004


In [64]:
geocode(df.iloc[0,:])

''

In [69]:
print(len(df))
print(len(df[df['county'] !='']))

1309
1302


In [74]:
df['county'].unique()

array(['', 'Miami County', 'Franklin County', 'Clearfield County',
       'Delaware County', 'Union County', 'Greene County',
       'Fairfield County', 'Summit County', 'Baruta', 'Montgomery County',
       'Clay County', 'Pinal County', 'Forsyth County', 'Yakima County',
       'Prowers County', 'Wood County', 'Boone County', 'Raigarh',
       'Second District NCR', 'Campbell County', 'Fourth District NCR',
       'Kent', 'Cuyahoga County', 'Bartholomew County', 'Scioto County',
       'Somerset', 'Allegheny County', 'Provincia de Zaragoza',
       'Hocking County', 'Licking County', 'Hamilton', 'Solano County',
       'Lycoming County', 'Hinds County', 'Santa Clara County',
       'Honolulu', 'Ekurhuleni', 'Clark County', 'Cache County',
       'Wake County', 'Първомай', 'Reno County', 'Lancashire',
       'Ocean County', 'Hamilton County', 'Ingham County', 'Luna County',
       'Марица', 'Pemiscot', 'Champaign County', 'Benton County',
       'Blaine County', 'City of Charlottesvil

In [73]:
df_processed = df[df['county'].isin(['Marion County','Logan County', 'Union County', 'Madison County', 'Fayette County', 'Delaware County', 'Morrow County', 'Franklin County','Pickaway County', 'Ross County', 'Knox County', 'Licking County', 'Fairfield County', 'Perry County', 'Hocking County'])]
print(len(df_processed))

1234


In [76]:
df_processed['county'].unique()

array(['Franklin County', 'Delaware County', 'Union County',
       'Fairfield County', 'Hocking County', 'Licking County',
       'Logan County', 'Madison County', 'Marion County'], dtype=object)

In [78]:
df_processed.head()

,Industry,BusinessName,Id,FirstName,Email,LastName,Address,City,State,Zip,EnrollmentDate,county
2,office_and_education,MAPFRE Stadium,10407,Jordan,jenke@columbuscrewsc.com,Enke,1%Black%and%Gold%Blvd,Columbus,OH,43211,2015-04-21,Franklin County
3,restaurant_and_grocery,Cathy Tiffany Salon Lofts,9446,Cathy,cathy@cathytiffany.com,Tiffany,1%E.%Campus%View%Blvd,Columbus,OH,43235,2008-08-02,Franklin County
4,office_and_education,Nationwide Insurance,9728,Kelly,grosenk1@nationwide.com,Grosenbacher,1%Nationwide%Plaza,Columbus,OH,43215,2009-10-19,Franklin County
5,office_and_education,American Electric Power,10281,Dave,dascott@aep.com,Scott,"1%Riverside%Plaza,%10th%Floor",Columbus,OH,43215,2012-12-19,Franklin County
7,restaurant_and_grocery,Phat Wraps,9476,jose,jbecerra8@aol.com,Becerra,10%east%12th%Avenue,Columbus,OH,43201,2009-01-19,Franklin County


In [80]:
df_processed['year'] = pd.to_datetime(df_processed['EnrollmentDate']).dt.year

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [82]:
df_select = df_processed.loc[:,['year','county','Industry']].copy()
df_select.loc[:,'count'] = 1  
df_select

,year,county,Industry,count
2,2015,Franklin County,office_and_education,1
3,2008,Franklin County,restaurant_and_grocery,1
4,2009,Franklin County,office_and_education,1
5,2012,Franklin County,office_and_education,1
7,2009,Franklin County,restaurant_and_grocery,1
...,...,...,...,...
1303,2009,Franklin County,office_and_education,1
1304,2017,Franklin County,office_and_education,1
1305,2014,Franklin County,office_and_education,1
1306,2016,Franklin County,office_and_education,1


In [83]:
#region
grouped_region = df_select.groupby(['year','Industry'],as_index = False)['count'].sum()
region = grouped_region.pivot(index='year', columns='Industry', values='count').fillna(0)
region.reset_index(inplace = True)
region

Industry,year,industrial,office_and_education,restaurant_and_grocery
0,2008,49,111,52
1,2009,41,73,23
2,2010,22,45,8
3,2011,37,70,12
4,2012,16,56,8
5,2013,17,180,15
6,2014,19,42,16
7,2015,14,75,13
8,2016,17,68,18
9,2017,7,21,4


In [85]:
#county
grouped_county = df_select.groupby(['county','Industry','year'],as_index = False)['count'].sum()
county = grouped_county.pivot_table(index=['county','year'], columns='Industry', values='count').fillna(0)
county.reset_index(inplace = True)
county.head()

Industry,county,year,industrial,office_and_education,restaurant_and_grocery
0,Delaware County,2008,3.0,4.0,0.0
1,Delaware County,2009,2.0,2.0,0.0
2,Delaware County,2010,2.0,4.0,0.0
3,Delaware County,2011,2.0,3.0,0.0
4,Delaware County,2012,1.0,4.0,1.0


In [86]:
countycode = pd.read_excel('../Ohio_county_code.xlsx')
countycode.head()

,Geoid,County_name
0,39001,Adams
1,39003,Allen
2,39005,Ashland
3,39007,Ashtabula
4,39009,Athens


In [87]:
county['county'] = county.county.str.split(expand=True)[0]
county['county'].unique()

array(['Delaware', 'Fairfield', 'Franklin', 'Hocking', 'Licking', 'Logan',
       'Madison', 'Marion', 'Union'], dtype=object)

In [88]:
county_result = county.merge(countycode[['Geoid','County_name']],left_on = 'county',right_on='County_name',how = 'left')
county_result.head()

,county,year,industrial,office_and_education,restaurant_and_grocery,Geoid,County_name
0,Delaware,2008,3.0,4.0,0.0,39041,Delaware
1,Delaware,2009,2.0,2.0,0.0,39041,Delaware
2,Delaware,2010,2.0,4.0,0.0,39041,Delaware
3,Delaware,2011,2.0,3.0,0.0,39041,Delaware
4,Delaware,2012,1.0,4.0,1.0,39041,Delaware


In [91]:
#'39049', '39041', '39159', '39045', '39073', '39089', '39097','39101
county_result['Geoid'].unique()

array([39041, 39045, 39049, 39073, 39089, 39091, 39097, 39101, 39159],
      dtype=int64)

In [90]:
county_result.drop(columns=['county','County_name'],inplace = True)
county_result.head()

,year,industrial,office_and_education,restaurant_and_grocery,Geoid
0,2008,3.0,4.0,0.0,39041
1,2009,2.0,2.0,0.0,39041
2,2010,2.0,4.0,0.0,39041
3,2011,2.0,3.0,0.0,39041
4,2012,1.0,4.0,1.0,39041
